In [46]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests

In [47]:
import json

In [48]:
browser = Browser('chrome')

In [4]:
base_url = 'https://www.amazon.com'
product_page_url = f"{base_url}/s?i=specialty-aps&srs=17296221011&rh=n%3A17296221011&fs=true"

In [162]:
# HEADERS = ({'User-Agent':
#         'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
#         'Accept-Language': 'en-US,en;q=0.9'})

In [85]:
##image
def getImage(div):
    try:
        return div.find('img', {'class': 's-image'}).attrs['src']
    except:
        return ''

## returns https://m.media-amazon.com/images/I/71ok9jCkgVL._AC_UY218_.jpg

In [86]:
## title
def getTitle(div):
    try:
        return div.find('h2').text.strip()
    except:
        return ''

##returns Lessons in Chemistry: A Novel

In [87]:
## prod link
def getProdLink(div):
    return div.find('span', {'data-component-type': 's-product-image'}).find('a').attrs["href"]

## returns /Lessons-in-Chemistry-A-Novel/dp/B09BBK79VB/ref=sr_1_1?qid=1680240999&sr=8-1&srs=17296221011

In [106]:
## author
def getAuthor(div):
    try:
        author_cont = div.find('div', {'id': 'bylineInfo'}).find_all('span', {'class': 'author notFaded'})
        return [item.find('a').text.strip() for item in author_cont]
    
    except:
        return ''

## returns Bonnie Garmus, Miranda Raison, et al.

In [89]:
## rating
def getRating(div):
    try:
        return div.find('div', {'class': 'a-row a-size-small'}).find('span', {'class': 'a-size-base'}).text
    except:
        return '0'

## returns 4.6

In [90]:
## total reviews
def getReviewCount(div):
    try:
        return div.find('div', {'class': 'a-row a-size-small'}).find('span').next_sibling.attrs["aria-label"]
    except:
        return '0'

## returns 126,027

In [91]:
## publisher
def getPublisher(product_soup):
    try:
        return product_soup.find('div', {'id': 'rpi-attribute-audiobook_details-publisher'}).find(
            'div', {'class': 'rpi-attribute-value'}).text.strip()
    except:
        try:
            return product_soup.find('div', {'id': 'rpi-attribute-book_details-publisher'}).find(
                'div', {'class': 'rpi-attribute-value'}).text.strip()
        except:
            pass
    return ''

## returns Random House Audio

In [92]:
## publish date

def getPublishDate(product_soup):
#     print("-------------------")
    try:
        publication_container = product_soup.find_all('div', {'id': 'rpi-attribute-book_details-publication_date'})
#         print("    Found " + str(len(publication_container)) + " divs for publicatioon date")
        for item in publication_container:
            date = item.find('div', class_="a-section a-spacing-none a-text-center rpi-attribute-value").find('span').text.strip()
#         print(date)
        return date
    except Exception as e:
#         print("    Couldn't get publication date: " + str(e))
        try:
            audio_pub = product_soup.find_all('div', {'id': 'rpi-attribute-audiobook_details-release-date'})
#             print("    Found " + str(len(audio_pub)) + " divs for release date")
            for item in audio_pub:
                audio_date = item.find('div', class_="a-section a-spacing-none a-text-center rpi-attribute-value").find('span').text.strip()
#             print(audio_date)
            return audio_date
        except Exception as e:
#             print("    Couldn't get release date: " + str(e))
            pass
#     print("Publicatioon date not found")
    return 'Publication date not found'

##returns April 5, 2022

In [93]:
##prices
def getPrices(product_soup):
    try:
        prices_list = product_soup.find_all('li', { 'class': 'swatchElement' })
        prices = {}
        for p in prices_list:
            tokens = p.find('a').text.split()
            type, cost = tokens[0], tokens[-1]
            prices[type] = cost
        return prices
    except:
        return {}


##returns {'Kindle': '$14.99',
##         'Audiobook': '$0.00',
##         'Hardcover': '$17.99',
##         'Paperback': '$27.90',
##        'Spiral-bound': '$37.81'}

In [271]:
# breakdown for ratings
# 1, 2, 3, 4, 5

In [272]:
# def getRatingBreakdown(product_soup):
#     try:
#         return [p.text.strip()[:-1] for p in product_soup.find(
#             'table', {'id': 'histogramTable'}
#         ).find_all('span', { 'class': 'a-size-base' })[1::2][::-1]]
#     except:
#         return [0,0,0,0,0]

# ## return ['1', '1', '6', '20', '72']

In [273]:
## ranking
# def getRanking(product_soup):
#     ranking_tr = []
#     try:
#         ranking_tr = product_soup.find(
#             'span', {'id': 'audibleProductDetailsSalesRank'}
#         ).find('td').find('span').find_all('span')
#     except:
#         try:
#             ranking_tr = product_soup.find(
#                 'ul', {'class': 'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'}
#             ).find('ul').find_all('li')
#         except:
#             pass
#     return [t.text for t in ranking_tr]

##returns ['#5 in Audible Books & Originals (See Top 100 in Audible Books & Originals)',
##         '#1 in Historical Fiction (Books)',
##         '#1 in Historical Fiction (Audible Books & Originals)',
##         '#1 in Humorous Fiction & Satire']


In [94]:
def custReviews(product_soup):
    try:
        reviews = product_soup.find('div', {'class': 'a-section review-views celwidget'}).find_all('div', {'class': 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'})
    except:
        pass
    return [review.text.strip() for review in reviews]

In [82]:
pages = 7

In [101]:
result = []

In [111]:
for p in range(1, pages+1):
    page_url = f"{product_page_url}&page={p}"
    browser.visit(page_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html')
    book_items = soup.find_all("div", attrs={"data-component-type":'s-search-result'})
    
    for book in book_items:
        book_details  = {}
        book_details["image"] = getImage(book)
        book_details["title"] = getTitle(book)
        book_details["rating"] = getRating(book)
        book_details["review_count"] = getReviewCount(book)
        try:
            link = getProdLink(book)
            book_details["product_link"] = link
            product_url = f"{base_url}{link}"
            browser.visit(product_url)
            prod_html = browser.html
            product_soup = BeautifulSoup(prod_html, 'html')

            book_details["author"] = getAuthor(product_soup)
            book_details["publisher"] = getPublisher(product_soup)
            book_details["publish_date"] = getPublishDate(product_soup)
            book_details["prices"] = getPrices(product_soup)
            book_details["reviews"] = custReviews(product_soup)
        except:
            print("couldnt load product page")
            pass
        result.append(book_details)
        print(book_details["title"])

-------------------
Lessons in Chemistry: A Novel
-------------------
Tomorrow, and Tomorrow, and Tomorrow: A novel
-------------------
Demon Copperhead: A Novel
-------------------
Remarkably Bright Creatures: A Novel
-------------------
Reminders of Him: A Novel
-------------------
Mad Honey: A Novel
-------------------
I'm Glad My Mom Died
-------------------
Horse: A Novel
-------------------
Fairy Tale
-------------------
Lighter: Let Go of the Past, Connect with the Present, and Expand the Future
-------------------
Finding Me: A Memoir
-------------------
Carrie Soto Is Back: A Novel
-------------------
An Immense World: How Animal Senses Reveal the Hidden Realms Around Us
-------------------
Our Missing Hearts: A Novel
-------------------
Friends, Lovers, and the Big Terrible Thing: A Memoir
-------------------
Surrender: 40 Songs, One Story
-------------------
The Marriage Portrait: A novel
-------------------
And There Was Light: Abraham Lincoln and the American Struggle
----

In [114]:
[r for r in result]

[{'image': 'https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/71ok9jCkgVL._AC_UY218_.jpg',
  'title': 'Lessons in Chemistry: A Novel',
  'rating': '4.6',
  'review_count': '129,563',
  'product_link': '/Lessons-in-Chemistry-A-Novel/dp/B09BBK79VB/ref=sr_1_1?qid=1680755080&sr=8-1&srs=17296221011',
  'author': ['Bonnie Garmus', 'Miranda Raison'],
  'publisher': 'Random House Audio',
  'publish_date': 'April 5, 2022',
  'prices': {'Kindle': '$14.99',
   'Audiobook': '$0.00',
   'Hardcover': '$17.99',
   'Paperback': '$27.90',
   'Spiral-bound': '$37.81'},
  'reviews': ["This book is like quicksand. It will suck you in and not let go until you finish it. Oh, it is GOOD!Written by Bonnie Garmus, this is a novel that is both a riveting story and a feminist manifesto. It's 1952 and Elizabeth Zott, a brilliant wannabe chemist, is being stifled at every turn, prevented from getting a Ph.D. (and when you find out the reason, you will want to throw something) and unable to find

In [115]:
f = open('top100_books_2022.json', 'w')

In [116]:
json.dump(result, f)

In [117]:
f.close()